我们最想要的数据格式就是表数据，但这表格并不是excel文件，而是存在于网页上的表数据。比如本教程实验网站

http://s.askci.com/stock/a
    
![](img/1.png)

一般遇到这种数据，最简单的方法就是复制粘贴，但是当
- 复制粘贴格式混乱
- 粘贴工作量太多
就需要写爬虫帮我们处理这件事情。但是常见的爬虫教程没有详细教大家怎么处理这种网址，今天我就简单说下办法。

### 审查网页
![](img/2.png)
我们可以发现
- 视图(平常肉眼所见)与html(图片下方的html)的一一对应对关系
- html中有``<table>``标签

### 网页种的table表

```html
<table class="..." id="...">
    <thead>
    <tr>
    <th>...</th>
    </tr>
    </thead>
    <tbody>
        <tr>
            <td>...</td>
        </tr>
        <tr>...</tr>
        <tr>...</tr>
        <tr>...</tr>
        <tr>...</tr>
        ...
        <tr>...</tr>
        <tr>...</tr>
        <tr>...</tr>
        <tr>...</tr>        
    </tbody>
</table>
```

先来简单解释一下上文出现的几种标签含义：

- ``<table>``	: 定义表格
- ``<thead>``	: 定义表格的页眉
- ``<tbody>``	: 定义表格的主体
- ``<tr>``	: 定义表格的行
- ``<th>``	: 定义表格的表头
- ``<td>``	: 定义表格单元


看到table这样的表格数据，我们可以利用pandas模块里的read_html函数方便快捷地抓取下来。

### 爬虫思路
1. 寻找表格网页对应的url的网址规律
2. 试图对某一个表格网页url进行访问
3. 定位到所想要的table部分,得到``<table>...</table>``字符串table_str
4. 使用pd.read_html(table_str),得到一个df
5. 循环2-4，得到的dfs(存放多个df的列表)
6. 多个df合并为新的alldf(alldf为dataframe)结果保存到csv文件中， alldf.to_csv(filename)

## 实战
### 1. 网址规律
先尝试点击翻页，看看网址栏中url变化
![](img/page2.png)
![](img/page3.png)
我们很幸运的发现网址前后有变化

```
http://s.askci.com/stock/a/0-0?reportTime=2019-09-30&pageNum=2#QueryCondition
http://s.askci.com/stock/a/0-0?reportTime=2019-09-30&pageNum=3#QueryCondition
```

所以网址规律

```
http://s.askci.com/stock/a/0-0?reportTime=2019-09-30&pageNum={pageNum}#QueryCondition
```

### 2. 尝试对任意页面(page2)进行访问
尝试对第二页进行访问，只要 **肉眼见到** 和 **html中的数据** 对应上，就证明访问是正常的。
![](img/page2req.png)
这里用 **深圳市特力(集团)股份有限公司**来验证

这次很幸运，因为requests.get(url)访问时并没有伪装成浏览器，网站也没有封锁这个python爬虫。我们继续

### 3. 定位table数据
![](img/定位.png)
如图，F12打开开发者工具， 我们定位到``id=myTable04``的table。


In [1]:
import requests

url = 'http://s.askci.com/stock/a/0-0?reportTime=2019-09-30&pageNum=6#QueryCondition'
resp = requests.get(url)
resp.text

'\r\n\r\n\r\n<!doctype html>\r\n<html>\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n    <title>A股上市公司名单-A股上市公司名录-A股上市公司大全-商业计划书-可研报告-中商产业研究院数据库</title>\r\n    <meta name="keywords" content="宏观数据,产量数据,销量数据,上市公司数据,行业数据,价格数据,中商产业研究院数据库" />\r\n    <meta name="description" content="宏观数据,产量数据,销量数据,上市公司数据,行业数据,价格数据,中商产业研究院数据库" />\r\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0;" />\r\n    <meta name="apple-mobile-web-app-capable" content="yes" /><!--开启WEB-APP程序的支持 全屏显示-->\r\n    <meta name="apple-mobile-web-app-status-bar-style" content="black" /><!--WEB-APP应用状态条颜色-->\r\n    <meta name="formt-detection" content="telephone=no" /><!--禁止数字自动识别为电话号码-->\r\n    <link href="http://jscss.askci.com/s_datajscs/css/pc_public.css" rel="stylesheet" type="text/css" /><!--公共样式表-->\r\n    <link href="http://jscss.askci.com/s_datajscs/css/header.css" rel="stylesheet" type=

In [2]:
from bs4 import BeautifulSoup
import pandas as pd

bsObj = BeautifulSoup(resp.text)
table = bsObj.find('table', {'id': 'myTable04'})
table

<table cellpadding="0" cellspacing="0" class="fancyTable" id="myTable04">
<thead>
<tr>
<th>序号</th>
<th>股票代码</th>
<th>股票简称</th>
<th>公司名称</th>
<th>省份</th>
<th>城市</th>
<th><div class="nav_hov"><a href="javascript:void(0)">主营业务收入(201909) <i class="icon_sxbottom"></i></a></div></th>
<th><div class="nav_hov"><a href="javascript:void(0)">净利润(201909)<i class="icon_sxbottom"></i></a></div></th>
<th>员工人数</th>
<th>上市日期</th>
<th>招股书</th>
<th>公司财报</th>
<th>行业分类</th>
<th class="text_left">产品类型</th>
<th class="text_left">主营业务</th>
</tr>
</thead>
<tbody>
<tr>
<td>101</td>
<td><a href="/stock/summary/000488/" target="_blank">000488</a></td>
<td><a href="/stock/summary/000488/" target="_blank">晨鸣纸业</a></td>
<td>山东晨鸣纸业集团股份有限公司</td>
<td>潍坊市</td>
<td>寿光市</td>
<td>220.14亿</td>
<td>11.18亿</td>
<td>14311</td>
<td>2000-11-20</td>
<td>--</td>
<td><a href="http://s.askci.com/stock/financialreport/000488/" target="_blank"><img src="http://image1.askci.com/s_dataimages/images/data-icon-2017-11-09000000017.png"/></

### 3. table_str2dataframe
使用pd.read_html(table_str)将table字符串变为dataframe

In [7]:
table_str = str(table)
df = pd.read_html(table_str)[0]
df.head()

,序号,股票代码,股票简称,公司名称,省份,城市,主营业务收入(201909),净利润(201909),员工人数,上市日期,招股书,公司财报,行业分类,产品类型,主营业务
0,101,488,晨鸣纸业,山东晨鸣纸业集团股份有限公司,潍坊市,寿光市,220.14亿,11.18亿,14311,2000-11-20,--,NaN,造纸,主营业务:机制纸及板纸和造纸原料、造纸机械、电力、热力的生产与销售,机制纸及板纸和造纸原料、造纸机械、电力、热力的生产与销售
1,102,498,山东路桥,山东高速路桥集团股份有限公司,济南市,市中区,157.68亿,5.10亿,8703,1997-06-09,--,NaN,路桥建设,主营业务:路桥工程施工与养护施工,路桥工程施工与养护施工
2,103,501,鄂武商A,武汉武商集团股份有限公司,武汉市,江汉区,128.81亿,8.18亿,9118,1992-11-20,--,NaN,百货,主营业务:商业零售及批发业务,商业零售及批发业务
3,104,502,绿景控股,绿景控股股份有限公司,广州市,海珠区,1187.37万,-194.11万,127,1992-11-23,--,NaN,房地产开发,主营业务:从事房地产开发及物业管理业务,从事房地产开发及物业管理业务
4,105,503,国新健康,国新健康保障服务集团股份有限公司,海口市,龙华区,5366.20万,-1.47亿,1223,1992-11-30,--,NaN,电子商务B2B,主营业务:医药电子商务、医疗福利管理等,医药电子商务、医疗福利管理等


跟我们设想的不太一样，结果不是dataframe应该有的样子。经过谷歌和百度，解决办法如下

In [9]:
df = pd.read_html(str(table),converters={'股票代码': str})[0]
df.head()

,序号,股票代码,股票简称,公司名称,省份,城市,主营业务收入(201909),净利润(201909),员工人数,上市日期,招股书,公司财报,行业分类,产品类型,主营业务
0,101,000488,晨鸣纸业,山东晨鸣纸业集团股份有限公司,潍坊市,寿光市,220.14亿,11.18亿,14311,2000-11-20,--,NaN,造纸,主营业务:机制纸及板纸和造纸原料、造纸机械、电力、热力的生产与销售,机制纸及板纸和造纸原料、造纸机械、电力、热力的生产与销售
1,102,000498,山东路桥,山东高速路桥集团股份有限公司,济南市,市中区,157.68亿,5.10亿,8703,1997-06-09,--,NaN,路桥建设,主营业务:路桥工程施工与养护施工,路桥工程施工与养护施工
2,103,000501,鄂武商A,武汉武商集团股份有限公司,武汉市,江汉区,128.81亿,8.18亿,9118,1992-11-20,--,NaN,百货,主营业务:商业零售及批发业务,商业零售及批发业务
3,104,000502,绿景控股,绿景控股股份有限公司,广州市,海珠区,1187.37万,-194.11万,127,1992-11-23,--,NaN,房地产开发,主营业务:从事房地产开发及物业管理业务,从事房地产开发及物业管理业务
4,105,000503,国新健康,国新健康保障服务集团股份有限公司,海口市,龙华区,5366.20万,-1.47亿,1223,1992-11-30,--,NaN,电子商务B2B,主营业务:医药电子商务、医疗福利管理等,医药电子商务、医疗福利管理等


### 4. 完整代码
重复操作

1. 根据url规律，对多个表格url网页进行批量访问
2. 重复之前对操作，得到dfs（存放多个df的列表）
3. 将dfs转化为名为alldf的dataframe，并alldf.to_csv

In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#伪装为浏览器
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36"}

#df列表容器，存放每个表格df
dfs = []

#一共有185个页面，这里仅仅就前10页进行测试
template = 'http://s.askci.com/stock/a/0-0?reportTime=2019-09-30&pageNum={pageNum}#QueryCondition'
for page in range(1, 11):
    print(page)
    url = template.format(pageNum=page)
    resp = requests.get(url, headers=headers)
    
    #定位table，得到table字符串
    bsObj = BeautifulSoup(resp.text)
    table_str = str(bsObj.find('table', {'id': 'myTable04'}))
    df = pd.read_html(table_str)[0]
    dfs.append(df)
    
#将dfs转为alldf，注意dfs是列表，而alldf是dataframe
alldf = pd.concat(dfs)
#输出为csv
alldf.to_csv('data.csv')

1
2
3
4
5
6
7
8
9
10


### 5. 采集结果
![](img/上市公司.gif)
### 参数学习
```python
pandas.read_html(io, header=None, skiprows=None, attrs=None, parse_dates=False, encoding=None, converters=None)
```
**常用的参数**：
- io: 实际上io这个位置不仅可以是table字符串，还可以是表格的url、表格的html文本、或者本地文件等；但建议爬在本案例中用table_str而不是url。网址不接受https，尝试去掉s后爬去
- header：标题所在的行数；
- skiprows：跳过的行；
- attrs：表格属性，对指定复合attrs值的表格进行解析，比如 ``attrs = {‘id’: ‘xxx’}``解析id为xxx的table
- parse_dates：是否解析日期，默认不解析
- encoding: 文件编码格式，常见的有 utf-8、 gbk等，默认为None
- converters: 字典格式，指定某一列按照某一类型数据去处理。如converters={'股票代码': str}，对股票代码这一列按照字符串(而不是数字)方式处理

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
dfs = []
template = 'http://s.askci.com/stock/a/0-0?reportTime=2019-09-30&pageNum={page}#QueryCondition'
for page in range(1, 6):
    url = template.format(page=page)
    resp = requests.get(url)
    bsObj = BeautifulSoup(resp.text)
    table_str = str(bsObj.find('table', {'id':'myTable04'}))
    df = pd.read_html(table_str)[0]
    dfs.append(df)
    
alldf = pd.concat(dfs)
alldf.to_csv('alldfdata.csv')